# Terraform
Things change over time

In [ ]:
#| default_exp terraform

In [ ]:
#| export
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple
from dataclasses import dataclass,  field, asdict
import json
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

#custom
import inspect
import copy

In [ ]:
from HexMagic.primitives import MapPath, MapSize, MapRect, MapCord 
from HexMagic.primitives import HexGrid, HexPosition ,  HexRegion , HexWrapper
from HexMagic.terrain import Terrain , TerraDemo
from HexMagic.terrainpatterns import TerrainPatterns
from HexMagic.climate import TerrainFactory
from HexMagic.hydrology import DrainageBasins
from HexMagic.styles import  StyleCSS 

## Some configs

In [ ]:
#| export
@dataclass
class SeismicEvent:
    kind: str  # e.g., "volcano", "erosion", "glacier", "meteor"
    name: str
    order_id: int
    properties: dict  # string key-value pairs
    adjustment: np.ndarray
    caller: str
    
    def encode(self) -> str:
        """Encode the seismic event to a string format."""
        ret = f"kind:{self.kind}\n"
        ret += f"name:{self.name}\n"
        ret += f"caller:{self.caller}\n"
        ret += f"order_id:{self.order_id}\n"
        
        # Encode properties as tab-separated key=value pairs
        prop_str = '\t'.join([f"{k}={v}" for k, v in self.properties.items()])
        ret += f"properties:{prop_str}\n"
        
        ret += "+adjustment:\n"
        # Encode the numpy array as tab-separated values
        ret += '\t'.join([f"{int(x)}" for x in self.adjustment]) + "\n"
        ret += "-adjustment:\n"
        
        return ret
    
    @staticmethod
    def decode(s: str) -> 'SeismicEvent':
        """Decode a string to create a SeismicEvent."""
        lines = s.strip().split('\n')
        
        kind = None
        name = None
        caller = None
        order_id = None
        properties = {}
        adjustment = None
        in_adjustment = False
        
        for line in lines:
            if in_adjustment:
                if line.startswith('-adjustment:'):
                    in_adjustment = False
                else:
                    # Parse the adjustment array
                    adjustment = np.array([float(x) for x in line.split('\t')])
            elif ':' in line:
                key, val = line.split(':', 1)
                key = key.strip()
                val = val.strip()
                
                if key == 'caller':
                    caller = val
                if key == 'kind':
                    kind = val
                elif key == 'name':
                    name = val
                elif key == 'order_id':
                    order_id = int(val)
                elif key == 'properties':
                    # Parse key=value pairs
                    if val:
                        pairs = val.split('\t')
                        for pair in pairs:
                            if '=' in pair:
                                k, v = pair.split('=', 1)
                                properties[k.strip()] = v.strip()
                elif key == '+adjustment':
                    in_adjustment = True
        
        return SeismicEvent(
            kind=kind,
            name=name,
            order_id=order_id,
            properties=properties,
            adjustment=adjustment,
            caller = caller
        )

In [ ]:
#| export
@dataclass
class ClimateRenderConfig:
    mode: str = "climate_zones"
    
    # Rivers
    show_rivers: bool = False
    river_max_count: int = 6
    river_branches: int = 2  # simplify() parameter
    river_color: str = "#1565c0"
    river_opacity: float = 0.7
    river_min_width: float = 1.0
    river_max_width: float = 8.0
    
    # Overlays
    show_elevation_borders: bool = True
    #show_coastline: bool = False
    show_legend: bool = True
    
    debug: bool = False

In [ ]:
#| export
@dataclass
class MapRenderConfig:
    show_hexes: bool = True
    show_coastline: bool = False
    animate: bool = False
    show_biomes: bool = False
    title: str = ""
    subtitle: str = ""
    description:str = ""
    icon:str = ""
    background_color: str = "#81b1e1ff"
    climate: ClimateRenderConfig = None
    shrink:int = 1  # was 'skrink'
    compass:str = None

In [ ]:
#| export
@dataclass
class VolcanoSettings:
    height: int = 300
    rings: int = 2
    variability= 80
    initial_threshold=0.4

In [ ]:
#| export
#This currently is under development

@dataclass
class LegendLabel:
    """used for labeling the legend"""
    icon: StyleCSS = None
    label: str = ""
    tip: str = ""
    group:int = 0

## Terraform

In [ ]:
#| export
class Terraform:
    def __init__(self, terrain: Terrain,config:MapRenderConfig=None):
        self.terrain = terrain
        self.events: List[SeismicEvent] = []
        # rivers placeholder for later
        self.plates: List[Plate] = []
        if config is None:
            rCofig = ClimateRenderConfig()
            self.config = MapRenderConfig(climate=rCofig)
        else:
            self.config = config

    @property
    def grid(self):
        return self.terrain.hexGrid

    @property
    def builder(self):
        return self.grid.builder



    @staticmethod
    def decode(s: str) -> 'Terraform':
        """Decode a string to create a Terraform."""
        lines = s.split('\n')
        
        terrain_lines = []
        event_lines = []
        config_lines = []
        current_section = None
        current_event = []
        plates = []
        
        i = 0
        while i < len(lines):
            line = lines[i]
            
            if line.startswith('+terrain:'):
                current_section = 'terrain'
            elif line.startswith('-terrain:'):
                current_section = None
            elif line.startswith('+configuration:'):
                current_section = 'configuration'
            elif line.startswith('-configuration:'):
                current_section = None
            elif line.startswith('+plate:'):
                current_section = 'plate'
            elif line.startswith('-plate:'):
                current_section = None
            elif line.startswith('+event:'):
                current_section = 'event'
                current_event = []
            elif line.startswith('-event:'):
                if current_event:
                    event_lines.append('\n'.join(current_event))
                current_event = []
                current_section = None
            elif current_section == 'terrain':
                terrain_lines.append(line)
            elif current_section == 'configuration':
                config_lines.append(line)
            elif current_section == 'event':
                current_event.append(line)
            elif current_section == 'plate':
                plates.append(line)
            
            i += 1
        
        # Decode terrain
        terrain_str = '\n'.join(terrain_lines)
        terrain = Terrain.decode(terrain_str)
        
        # Decode configuration
        config = None
        if config_lines:
            config_dict = json.loads('\n'.join(config_lines))
            # Handle nested ClimateRenderConfig
            climate_dict = config_dict.pop('climate', None)
            climate_config = ClimateRenderConfig(**climate_dict) if climate_dict else None
            config = MapRenderConfig(**config_dict, climate=climate_config)
        
        # Create Terraform
        terraform = Terraform(terrain, config=config)
        
        # Decode events
        for event_str in event_lines:
            event = SeismicEvent.decode(event_str)
            terraform.events.append(event)
        
        terraform.plates = [Plate.decode(x, terrain.hexGrid) for x in plates]
        return terraform

        
        
        # Create Terraform
        terraform = Terraform(terrain)

        
        # Decode events
        for event_str in event_lines:
            event = SeismicEvent.decode(event_str)
            terraform.events.append(event)
        
        terraform.plates = [Plate.decode(x,terrain.hexGrid) for x in plates]
        return terraform

In [ ]:
#| export
@patch
def encode(self: Terraform) -> str:
    """Encode the Terraform to a string format."""
    ret = "=== TERRAFORM ===\n"
    ret += "+terrain:\n"
    ret += self.terrain.encode()
    ret += "-terrain:\n"
    
    # Configuration section
    ret += "+configuration:\n"
    config_dict = asdict(self.config)
    ret += json.dumps(config_dict, indent=2) + "\n"
    ret += "-configuration:\n"
    
    ret += f"event_count:{len(self.events)}\n"

    if len(self.plates) > 0:
        ret += "+plate:\n"
        for plate in self.plates:
            ret += plate.encode()
        ret += "-plate:\n"
    
    for event in self.events:
        ret += "+event:\n"
        ret += event.encode()
        ret += "-event:\n"
    
    return ret

## Handle Seismic Events

In [ ]:
#| export
@patch
def add_event(self:Terraform, kind: str,  adjustment: np.ndarray,name: str=None, properties: dict={}):
    order_id = len(self.events)  # or use actual timestamp
    if name is None:
        name = f"event_{len(self.events)}"
    caller_name = ", ".join([x.function for x in inspect.stack()][1:3])
    event = SeismicEvent(kind, name, order_id, properties, adjustment,caller_name)
    self.events.append(event)

In [ ]:
#| export
@patch
def flatten(self: Terraform):
    """Flatten all events into a single terrain state."""
    # Start with initial terrain
    adjustment = np.zeros(len(self.terrain.elevations))
    
    # Apply all adjustments
    for event in self.events:
        adjustment += event.adjustment
    
    self.events = [SeismicEvent("flatten", "flatten", 0, {}, adjustment, "flatten()")]
    
    
    return self.terrain

In [ ]:
#| export
@patch
def final(self: Terraform):
    """Flatten all events into a single terrain state."""

    for i, event in enumerate(self.events):
        self.terrain.elevations += event.adjustment
        self.terrain.colorMap()
        self.grid.update()
        
        hex_layer_body = self.builder.layers[0].body
        layer_name = f"event_{i}_{event.name}"
        self.builder.adjust(layer_name, hex_layer_body)

    self.builder.adjust("root","")
    return self.builder

@patch
def element(self:Terraform,state:RingMenuState,wrapper:HexWrapper = None)->str:
    return self.final().xml()

## Rendering different layers

In [ ]:
#| export
@patch
def starterTerrain(demo:TerraDemo,debug = False):
    """Create a tropical island with three volcanoes and downsampled rivers."""
    
    # 1. Create blank ocean world with tropical preset
    bounds = MapRect(MapCord(0, 0), MapSize(800, 800))
    terrain , plates = TerrainFactory.create_ocean_world(
        bounds=bounds,
        preset='tropical',
        radius=15,
        lon_span=5.0,
        num_plates=16,
        ocean_fraction=0.6,
        debug = debug
    )

    mountains = terrain.find_peaks(15,4)
    for i , epicenter in enumerate(mountains):
        terrain.elevations += terrain.volcano(center=epicenter, adjusted=20+ ((i+1)*30), num_rings=5)

    if debug:
        print("\n=== COMPUTING CLIMATE ===")
    terrain.climate.configure(terrain,debug=debug)
    return Terraform(terrain)
    
    



In [ ]:
demoT = TerraDemo().starterTerrain()
demoT.terrain.colorMap()
demoT.terrain.hexGrid.update()
demoT.terrain.hexGrid.builder.show()

Initial seeds: 25 (spacing=11)
After elimination: 19
Final unique seeds: 19


In [ ]:
#| export
@patch
def generate_legend(self: Terraform, labels:[LegendLabel], config: ClimateRenderConfig):
    return ""

In [ ]:
#| export
@patch
def generate_climate_legend(self: Terrain , config: ClimateRenderConfig)->[LegendLabel]:
    return []

In [ ]:
#| export
@patch
def render_climate_zones(self: Terrain, config: ClimateRenderConfig) -> tuple[str, [LegendLabel]]:
    """
    Render climate zones with precipitation density overlay.
    
    Returns:
        (layer_svg, legend_svg) - two separate SVG strings
    """
    if 'climate' not in self.fields or 'precipitation' not in self.fields:
        # Compute if needed
        if hasattr(self, 'climate') and self.climate is not None:
            self.climate.configure(self, force_recompute=False)
        else:
            raise ValueError("Need climate data - use TerrainFactory.create_ocean_world()")
    
    # Generate the main layer
    layer_svg = self.dottedClimate()
    
    
    # Add elevation borders if requested
    if config.show_elevation_borders:
        layer_svg += self.elevation_borders()
    
    # Add coastline if requested
    #if config.show_coastline:
    #    layer_svg += self.coastline_svg()
    
    # Generate legend
    legend = []
    if config.show_legend:
        legend.extend( self.generate_climate_legend(config))
    
    return layer_svg, legend

In [ ]:
demoT.terrain.hexGrid.builder.layers = []
cliRC = ClimateRenderConfig()
rczText, leg = demoT.terrain.render_climate_zones(cliRC)
demoT.terrain.hexGrid.builder.adjust("render_climate_zones",rczText)
demoT.terrain.hexGrid.builder.show()


In [ ]:
#| export
@patch
def render_hydration(self: Terrain, config: ClimateRenderConfig) -> tuple[str, list[LegendLabel]]:
    """
    Render hydration layer with watershed boundaries, flow gradient, and elevation borders.
    
    Returns:
        (layer_svg, legend_labels)
    """
    basin = DrainageBasins(self, debug=config.debug)
    
    layer_svg = ""
    
    # Flow gradient (direction with gradient-based widths)
    #layer_svg += basin.gradient_overlay()

    layer_svg += basin.dotted_watershed_overlay()
    
    # Watershed boundaries
    #layer_svg += basin.boundary_overlay()
    
    # Elevation borders
    if config.show_elevation_borders:
        layer_svg += self.elevation_borders()
    
    # Legend
    legend = []
    #if config.show_legend:
    #    legend = self.generate_hydration_legend(config)
    
    return layer_svg, legend

In [ ]:
demoT.terrain.hexGrid.builder.layers = []
cliRC = ClimateRenderConfig()
rczText, leg = demoT.terrain.render_hydration(cliRC)
demoT.terrain.hexGrid.builder.adjust("render_climate_zones",rczText)
demoT.terrain.hexGrid.builder.show()

In [ ]:
#| export
@patch
def render_weather(self: Terrain, config: ClimateRenderConfig) -> tuple[str, list[LegendLabel]]:
    """
    Render hydration layer with watershed boundaries, flow gradient, and elevation borders.
    
    Returns:
        (layer_svg, legend_labels)
    """
    
    layer_svg = ""
    
    # Flow gradient (direction with gradient-based widths)
    #layer_svg += basin.gradient_overlay()

    layer_svg += self.render_icon_temperature()
    
    
    # Legend
    legend = []
    #if config.show_legend:
    #    legend = self.generate_hydration_legend(config)
    
    return layer_svg, legend

In [ ]:
demoT.terrain.hexGrid.builder.layers = []
cliRC = ClimateRenderConfig()
rczText, leg = demoT.terrain.render_weather(cliRC)
demoT.terrain.hexGrid.builder.adjust("render_climate_zones",rczText)
demoT.terrain.hexGrid.builder.show()

In [ ]:
#| export
@patch
def render_climate(self: Terraform,terrain, event_index: int) -> tuple[str, str]:
    """Render climate-based visualization including rivers."""
    config = self.config.climate
    
    
    legend = []
    ret = ""
    

    
    # Climate zones layer
    if config.mode == "climate_zones":
        climate_layer, legend = terrain.render_climate_zones(config=config)
        ret += climate_layer

    if config.mode == "hydro":
        hydro_layer, legend = terrain.render_hydration(config=config)
        ret += hydro_layer

    if config.mode == "weather":
        weather_layer, legend = terrain.render_weather(config=config)
        ret += weather_layer
    


    # Rivers via watershed system
    if config.show_rivers:
        basin = DrainageBasins(terrain, debug=config.debug)
        rivers = basin.get_major(config.river_max_count)
        
        river_svg = ""
        for river in rivers:
            river_svg += river.simplify(config.river_branches).draw(
                color=config.river_color,
                opacity=config.river_opacity,
                min_width=config.river_min_width,
                max_width=config.river_max_width
            )
        ret += river_svg
    
    # Legend
    legend_svg = self.generate_legend(legend, config)
    
    return ret, legend_svg

In [ ]:
@patch
def render_layer(self: Terraform, event_index: int, debug=False) -> str:
    """Render complete layer content based on terrain state and config.
    
    Args:
        event_index: Which event we're rendering (-1 for base, 0+ for events)
        config: Rendering configuration
    
    Returns:
        Complete layer content as string
    """
    config = self.config
    layer_content = ""

        # Ensure climate data is computed
    if not hasattr(self.terrain, 'climate') or self.terrain.climate is None:
        raise ValueError("Terrain must have climate configured")
    
    self.terrain.climate.configure(self.terrain,force_recompute=True)
    if debug:
        print(" --- render_layer ---")
        print(config)
    
    smaller = self.terrain # we will revisit scaling down
    

    
    # Add background rect to prevent seeing through to previous layers
    bg_style = StyleCSS("layer_bg", fill=config.background_color)
    self.builder.add_style(bg_style)
    layer_content += f'\t<rect width="{smaller.hexGrid.builder.width}" height="{smaller.hexGrid.builder.height}" class="{bg_style.name}"/>\n'
    if debug:
        print(" --- render_layer ---")
        print(config)
    
    
    # Add hexes if requested
    if config.show_hexes:
        if debug:
            print("Showing hexes")
        for i, hex in enumerate(smaller.hexGrid.hexes):
            layer_content += "\t" + hex.svg() + "\n"
    
    # Add coastline if requested
    if config.show_coastline:
        if debug:
            print("Showing show_coastline")
        # Find all land hexes (elevation > 0)
        land_hexes = set()
        for i, elevation in enumerate(smaller.elevations):
            if elevation > 0:
                land_hexes.add(i)
        
        if land_hexes:
            region = HexRegion(hexes=land_hexes, hex_grid=smaller.hexGrid)
            coastline_style = StyleCSS(
                "coastline", 
                fill="none", 
                stroke="#2c5f2d", 
                stroke_width=3
            )
            paths = region.trace_perimeter(style=coastline_style)
            
            self.builder.add_style(coastline_style)
            for path in paths:
                if path.points[0].distance(path.points[-1]) < 40:
                    cl = path.closed()
                else:
                    cl = path
                curved = cl.make_windy(iterations=1, offset_factor=0.1, seed=42)
                smooth = curved.smooth(iterations=1)
                layer_content += "\t" + smooth.svg() + "\n"
    
    # Add rivers if requested (placeholder for now)
    if config.climate is not None:
        
        layer, legend = self.render_climate(smaller, event_index)
        if debug:
            print(f"Showing climats size {len(layer)}")
        layer_content += layer


    # begin paste
    
    
    
    
    return layer_content

In [ ]:
demoT.terrain.hexGrid.builder.layers = []

mode="climate_zones"

# maui.add_climate_overlay(layer_name="root")
rCong = ClimateRenderConfig()
rCong.show_rivers = True
rCong.river_colors = ["#1976d2"]  # Default blue
rCong.show_elevation_borders = True
rCong.show_coastline  = True
rCong.mode = mode

mapC = MapRenderConfig(
title = "An Example Map",
subtitle = "Of us",
background_color = "white" ,
show_coastline  = True,
show_hexes = False,
climate = rCong)

demoT.config = mapC

#wrapper = HexWrapper(callBack=HexWrapper.route())

mapLayer = demoT.render_layer(1)
demoT.terrain.builder.adjust("mapLayer",mapLayer)
demoT.terrain.builder.show()

In [ ]:
@patch
def animation_loop(self: Terraform, 
                   fade_duration: float = 1.0, 
                   hold_duration: float = 2.0, 
                   reset_fade_duration: float = 0.5,
                   reset_hold_duration: float = 1.0,
                wrapper: HexWrapper = None,
                   debug: bool = False) -> SVGBuilder:
    """Generate an animated SVG that cycles through terrain changes repeatedly."""
    
    # Default config if none provided
    
    
    num_events = len(self.events)
    
    if num_events == 0:
        self.terrain.colorMap()
        self.terrain.hexGrid.update()
        return self.terrain.hexGrid.builder
    
    # Generate initial terrain as base layer
    saveOrg = self.terrain.elevations.copy()
    
    self.terrain.colorMap()
    self.terrain.hexGrid.update()
   
    # Calculate total cycle duration
    build_up_time = num_events * (fade_duration + hold_duration)
    final_hold_time = hold_duration
    reset_time = reset_fade_duration + reset_hold_duration
    
    total_cycle_time = build_up_time + final_hold_time + reset_time
    
    if debug:
        print(f"Total events: {num_events}")
        print(f"Build-up time: {build_up_time}s")
        print(f"Final hold: {final_hold_time}s")
        print(f"Reset time: {reset_time}s")
        print(f"Total cycle: {total_cycle_time}s")
    
    # Create layers for each event
    event_layers = []
    
    for i, event in enumerate(self.events):
        self.terrain.elevations += event.adjustment
        self.terrain.colorMap()
        #self.terrain.hexGrid.update()
        
        # Use render_layer to get complete layer content
        layer_body = self.render_layer(i)
        layer_name = f"event_{i}_{event.name}"
        self.builder.adjust(layer_name, layer_body)
        event_layers.append(layer_name)
        
        # Calculate when this layer fades in
        fade_in_start = i * (fade_duration + hold_duration)
        
        # This layer stays visible until the reset phase
        time_visible_until_reset = build_up_time + final_hold_time - fade_in_start
        
        # Create looping animation for this layer
        anim_id = f"loop_{layer_name}_{self.builder.animation_counter}"
        self.builder.animation_counter += 1
        
        # Calculate percentages for keyframes
        fade_in_pct = (fade_duration / total_cycle_time) * 100
        hold_until_reset_pct = ((fade_in_start + time_visible_until_reset) / total_cycle_time) * 100
        fade_out_pct = ((fade_in_start + time_visible_until_reset + reset_fade_duration) / total_cycle_time) * 100
        
        if debug:
            print(f"Layer {i} ({event.name}): fade_in at {fade_in_start:.1f}s, visible until {fade_in_start + time_visible_until_reset:.1f}s")
        
        # Keyframes: start hidden → fade in → stay visible → fade out → stay hidden → repeat
        keyframes = f"""
@keyframes {anim_id} {{
    0% {{ opacity: 0; }}
    {(fade_in_start / total_cycle_time) * 100:.1f}% {{ opacity: 0; }}
    {((fade_in_start + fade_duration) / total_cycle_time) * 100:.1f}% {{ opacity: 1; }}
    {hold_until_reset_pct:.1f}% {{ opacity: 1; }}
    {fade_out_pct:.1f}% {{ opacity: 0; }}
    100% {{ opacity: 0; }}
}}"""
        
        selector = f'g[data-layer="{layer_name}"]'
        
        rule = f"""
{selector} {{
    animation-name: {anim_id};
    animation-duration: {total_cycle_time}s;
    animation-timing-function: ease-in-out;
    animation-iteration-count: infinite;
    opacity: 0;
}}

{keyframes}"""
        
        if not hasattr(self.builder, 'custom_animations'):
            self.builder.custom_animations = []
        self.builder.custom_animations.append(rule)
    
    # Restore original terrain
    self.terrain.elevations = saveOrg
    
    return self.builder

NameError: name 'SVGBuilder' is not defined

In [ ]:
@patch
def render(self: Terraform,  wrapper: HexWrapper = None):
    """Render terraform based on RingMenuState configuration."""

    
    if self.config.animate:
        self.animation_loop(wrapper=wrapper)
    else:

        saveOrg = self.terrain.elevations.copy()
        self.terrain.colorMap()
        for i, event in enumerate(self.events):
            self.terrain.elevations += event.adjustment
        self.terrain.climate.configure(self.terrain)
        climateTex = self.render_layer(0)
        self.builder.adjust("climate",climateTex)
        if wrapper is not None:
            self.terrain.colorMap()
            self.terrain.hexGrid.update(wrapper)
            self.terrain.hexGrid.builder.layers[-1].set_opacity(0.2)
            #self.builder.adjust("touch",tex)
            #print(f"text {tex}")

        self.terrain.elevations = saveOrg

    # Add title and subtitle
    title_elements = ""
    config = self.config
    if self.config.title:
        title_text = config.title 
        title_style = StyleCSS(
            "map-title",
            fill="#2c3e50",
            font_size="24px",
            font_weight="bold",
            font_family="serif",
            text_anchor="middle"
        )
        self.builder.add_style(title_style)
        
        # Position title at top center
        title_x = self.builder.width / 2
        title_y = 30
        title_elements += f'<text x="{title_x}" y="{title_y}" class="map-title">{title_text}</text>\n'
    
    if self.config.subtitle:
        subtitle_text = self.config.subtitle
        subtitle_style = StyleCSS(
            "map-subtitle",
            fill="#7f8c8d",
            font_size="16px",
            font_style="italic",
            font_family="serif",
            text_anchor="middle"
        )
        self.builder.add_style(subtitle_style)
        
        # Position subtitle below title
        subtitle_x = self.builder.width / 2
        subtitle_y = 55
        title_elements += f'<text x="{subtitle_x}" y="{subtitle_y}" class="map-subtitle">{subtitle_text}</text>\n'
  
    self.builder.adjust("title_l",title_elements)

    return

NameError: name 'HexWrapper' is not defined